In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
%cd /content/drive/MyDrive/YLP

/content/drive/MyDrive/YLP


In [13]:
from RKNet import RKNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
import os
import time
import datetime

In [14]:
batch_size = 250

shift = transforms.Normalize(
    mean=[0.5,0.5,0.5],
    std=[1.,1.,1.],
  )

# Load standard images for control purposes
transformTest = transforms.Compose(
    [transforms.ToTensor(),
     shift])


testSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transformTest)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batch_size,
                                          shuffle=False)

eps = 0.05

# Perturb testing images
hueTransform = transforms.Compose(
[lambda x: transforms.functional.adjust_hue(x,hue_factor=eps),
  transforms.ToTensor(),
  shift]
)

hueSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=hueTransform)
hueLoader = torch.utils.data.DataLoader(hueSet, batch_size=batch_size,
                                        shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [15]:
def imshow(img):
  img = img / 2 + 0.5
  npimg = img.cpu().numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()

In [16]:
# Load trained network
PATH = os.path.join('results','current_resnet_20220319_0711.pt')
# Modify output layer before loading trained weights
net = torchvision.models.resnet18(pretrained=False)
net.fc = nn.Linear(512,10)
net.load_state_dict(torch.load(PATH))
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
# Choose device and load model
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

In [19]:
import csv

In [20]:

start = time.time()
sStartTime = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
file_name = 'norm_diff_resnet' + sStartTime + '.csv'

csv_path = os.path.join('results',file_name)


header = ['eps','y0_diff','y_diff']
with open(csv_path,'w') as f:
  writer = csv.writer(f)
  writer.writerow(header)
  eps = 0.3

  hueTransform = transforms.Compose(
    [lambda x: transforms.functional.adjust_hue(x,hue_factor=eps),
      transforms.ToTensor(),
      shift]
  )

  hueSet = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=hueTransform)
  hueLoader = torch.utils.data.DataLoader(hueSet, batch_size=batch_size,
                                            shuffle=False)

  # iterate over the mini-batches of normal and perturbed inputs together
  for data, data_tild in zip(testLoader,hueLoader):
    images = data[0]
    images_hue = data_tild[0]

    images = images.to(device)
    images_hue = images_hue.to(device)
    
    #Pass both batches of images through network
    outputs = net(images)
    outputs_hue = net(images_hue)
    for i in range(len(images)):
      # Find initial values
      y0 = images[i]
      y0_tild = images_hue[i]
      
      # Find final values
      y = outputs[i]
      y_tild = outputs_hue[i]

      # Take norms and save
      norm_init = torch.linalg.norm(y0-y0_tild)
      norm_final = torch.linalg.norm(y-y_tild)
      writer.writerow([eps,norm_init.item(),norm_final.item()])

Files already downloaded and verified
